Based on Faron's script that was adopted in the following notebook: https://www.kaggle.com/tunguz/another-xgb-allstate-starter/

In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-24T10:20:19.242385-04:00

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.10.0

Compiler    : Clang 14.0.3 (clang-1403.0.22.14.1)
OS          : Darwin
Release     : 22.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [19]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc
import xgboost as xgb

In [20]:
%watermark --iversions

pandas : 2.0.0
numpy  : 1.23.5
xgboost: 1.7.5



In [5]:
ID = 'id'
TARGET = 'loss'
SEED = 0
DATA_DIR = "../input"

TRAIN_FILE = f"{DATA_DIR}/train.csv.zip"
TEST_FILE = f"{DATA_DIR}/test.csv.zip"
SUBMISSION_FILE = f"{DATA_DIR}/sample_submission.csv.zip"


In [7]:
train = pd.read_csv(TRAIN_FILE)
test = pd.read_csv(TEST_FILE)

y_train = train[TARGET].ravel()

train.drop([ID, TARGET], axis=1, inplace=True)
test.drop([ID], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")
      
ntrain = train.shape[0]

(188318, 130),(125546, 130)


In [8]:
train_test = pd.concat((train, test)).reset_index(drop=True)

features = train.columns

cats = [feat for feat in features if 'cat' in feat]
for feat in cats:
    train_test[feat] = pd.factorize(train_test[feat], sort=True)[0]

print(train_test.head())

x_train = np.array(train_test.iloc[:ntrain,:])
x_test = np.array(train_test.iloc[ntrain:,:])

print(f"{train.shape},{test.shape}")


   cat1  cat2  cat3  cat4  cat5  cat6  cat7  cat8  cat9  cat10  ...     cont5   
0     0     1     0     1     0     0     0     0     1      0  ...  0.310061  \
1     0     1     0     0     0     0     0     0     1      1  ...  0.885834   
2     0     1     0     0     1     0     0     0     1      1  ...  0.397069   
3     1     1     0     1     0     0     0     0     1      0  ...  0.422268   
4     0     1     0     1     0     0     0     0     1      1  ...  0.704268   

      cont6     cont7    cont8    cont9   cont10    cont11    cont12   
0  0.718367  0.335060  0.30260  0.67135  0.83510  0.569745  0.594646  \
1  0.438917  0.436585  0.60087  0.35127  0.43919  0.338312  0.366307   
2  0.289648  0.315545  0.27320  0.26076  0.32446  0.381398  0.373424   
3  0.440945  0.391128  0.31796  0.32128  0.44467  0.327915  0.321570   
4  0.178193  0.247408  0.24564  0.22089  0.21230  0.204687  0.202213   

     cont13    cont14  
0  0.822493  0.714843  
1  0.611431  0.304496  
2  0.195

In [12]:
%%time
gbdt = XGBRegressor()
gbdt.fit(x_train, y_train)

CPU times: user 1min 34s, sys: 43.2 s, total: 2min 17s
Wall time: 8.3 s


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [15]:
submission = pd.read_csv(SUBMISSION_FILE)
submission.iloc[:, 1] = gbdt.predict(x_test)
submission.to_csv('../submissions/xgb_starter_0_sub.csv.zip', index=None, compression='zip')
submission.head()

,id,loss
0,4,1762.236450
1,6,2064.845703
2,9,10855.864258
3,12,6631.281250
4,15,1015.801880


Private score: 1188.66993, Public Score: 1177.1932

In [35]:
train_oof = np.zeros((x_train.shape[0], ))
test_preds = 0
train_oof.shape

(188318,)

In [36]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(x_train)):
    print("Fitting fold", jj+1)
    train_features = x_train[train_index]
    train_target = y_train[train_index]
    
    val_features = x_train[val_index]
    val_target = y_train[val_index]
    
    model = XGBRegressor()
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred
    print("Fold mean absolute error:", mean_absolute_error(val_target, val_pred))
    test_preds += model.predict(x_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold mean absolute error: 1197.6749061150742
Fitting fold 2
Fold mean absolute error: 1199.1418021613943
Fitting fold 3
Fold mean absolute error: 1201.0596971990026
Fitting fold 4
Fold mean absolute error: 1210.5617474790229
Fitting fold 5
Fold mean absolute error: 1199.8798760139027
CPU times: user 6min 30s, sys: 3min 25s, total: 9min 56s
Wall time: 36.6 s


In [37]:
print("Total mean absolute error:", mean_absolute_error(y_train, train_oof))

Total mean absolute error: 1201.663568014965


In [38]:
submission.iloc[:, 1] = test_preds
submission.to_csv(f'../submissions/xgb_starter_{n_splits}fold_0_sub.csv.zip', index=None, compression='zip')
submission.head()

,id,loss
0,4,1744.097290
1,6,2069.760986
2,9,9514.549805
3,12,6380.024414
4,15,908.304016


Private score: 1173.89776, Public Score: 1162.76837